In [6]:
import requests
from bs4 import BeautifulSoup
import csv
import collections
import re
import nltk
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
#From Github; how to replace and expand contactions
import re
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text.lower())

In [93]:
import requests
from bs4 import BeautifulSoup
import re


reviews = []
ratings = [] 

neg_reviews = []
pos_reviews = []

review_corpus = []


headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}


base_url = 'http://www.influenster.com/reviews/jcrewcom'

while True:
    
    
    r = requests.get(base_url,headers=headers)

    soup = BeautifulSoup(r.text, 'html.parser')
    
    review_divs = soup.findAll('div', attrs={'itemprop': re.compile("^reviewBody")})

    for review in review_divs:
        if len(review.contents) > 0:
#             print(review.contents[0].strip().lower())
            try:
                review = review.contents[0].strip().lower()
                review_corpus.append([review])
                #expandContractions will put the dictionary made earlier to replace the contractions in the reviews
                #Make sure to to run the cList dict cell or else there will be an error
                expandContractions(review)
                #Cleaning the lemmas or words in reviews now will make it easier when we start predictive modeling
                words = word_tokenize(review)
                words = word_tokenize(review.replace('\n',' '))
                clean_words = [word.lower() for word in words if word not in set(string.punctuation)]
                characters_to_remove = ["''",'``','...']
                clean_words = [word for word in clean_words if word not in set(characters_to_remove)]
                english_stops = set(stopwords.words('english'))
                clean_words = [word for word in clean_words if word not in english_stops]
                wordnet_lemmatizer = WordNetLemmatizer()
                lemma_list = [wordnet_lemmatizer.lemmatize(word) for word in clean_words]
                print(lemma_list)
                reviews.append(lemma_list)
            except:
                continue
        #Here we are using a simple control flow to recode the ratings for our model. If rating is greater than 3 positive, else negative   
    
    rating = soup.find_all('div', attrs={'itemprop': 'ratingValue' , 'content': re.compile("^[0-9]+$")})
    #the rating is actually an image, so we need to convert it into a string and then to an integer
    for rate in rating:
        if len(re.split('[<> ]',str(rate))) > 2:
            spl = re.split('[<> ]',str(rate))
#                 print(re.split('[=""]',spl[2]))
            int_rating = int(re.split('[=""]',str(rate))[2])
            print(int_rating)
            ratings.append(int_rating)
                
    
    next_page = soup.findAll('link', attrs={'rel': re.compile("^next")})
    print(next_page)
    if next_page is None or len(next_page) == 0:
        base_url = None
        break
    elif len(next_page) > 0:
        base_url = next_page[0].get('href')
    else:
        base_url = None
    
    if base_url is None:
        break
        
for int_rating, review in zip(ratings, reviews):        
    if int_rating == 1 or int_rating == 2 or int_rating == 3:
        rating = 'neg'
        ratings.append('neg')
        neg_reviews.append(review)
    elif int_rating == 4 or int_rating == 5:
        rating = 'pos'
        ratings.append('pos')
        pos_reviews.append(review)

my_df = pd.DataFrame(review_corpus)

my_df.to_csv('revised_jcrew_reviews.csv', index=False, header=False)

['j.crew', 'one', 'absolute', 'favorite', 'brand', 'since', 'starting', 'grad', 'school', 'last', 'year', '’', 'invest', 'lot', 'professional', 'clothing', 'love', 'style', 'j.', 'crew', 'especially', 'classic', 'piece', 'everyone', 'need', 'staple', 'also', 'love', 'factory', 'store', 'casual', 'clothes']
['clothes', 'gotten', 'really', 'boring', 'weirdly', 'baggy', 'still', 'love', 'business', 'sheath', 'dress', 'suiting', 'website', 'always', 'primary', 'way', 'purchasing', 'always', 'good', 'experience', 'p.', 'customer', 'service', 'always', 'great']
['j', 'crew', 'one', 'favorite', 'store', 'clothing', 'item', 'cute', 'classy', 'also', 'made', 'well', 'hold', 'wear', 'wash', 'little', 'pricier', 'side', 'worth', 'also', 'good', 'deal']
['love', 'jcrew', '’', 'online', 'store', 'many', 'option', 'choose', 'ship', 'super', 'fast', '90', 'closet', 'full', 'clothes', 'almost', 'always', 'find', 'size', 'wear', 'website', 'happy', 'online', 'store']
['amazing', 'clothing', 'love', 'sh

['cute', 'clothes', 'accessory', 'love', 'clothes', "'s", 'affordable', 'pretty', 'dressy', 'clothes', 'love', 'coat', 'rain', 'coat', 'got', 'bunch', 'stuff', 'christmas', 'would', 'recommend', 'great', 'store']
['like', 'jcrew', 'website', 'part', 'website', 'design', 'fresh', 'clean', '5', 'flat', 'rate', 'shipping', 'great', 'almost', 'always', 'sale', 'going', 'selection', 'incredibly', 'vast']
['love', 'looking', 'website', 'want', 'deal', '’', 'pas', 'always', 'extra', 'percent', 'sale', 'item', 'find', 'something', 'like', 'wait', 'day', 'sale']
['’', 'addicted', 'jcrew', 'jewelry', '’', 'big', 'big', 'problem', 'check', 'site', 'without', 'checking', 'without', 'le', '3-6', 'pair', 'earring', 'always', 'unique', 'beautiful', 'love', 'style', 'quality']
['jcrew', 'must', 'closet', 'piece', 'great', 'quality', 'affordable', 'especially', 'great', 'sale', 'employee', 'always', 'helpful', 'never', 'problem', 'ordering', 'online']
['recently', 'bought', 'first', 'pair', 'matchstick

['love', 'love', 'love', 'even', 'better', 'jcrewfactory.com', 'really', 'great', 'staple', 'piece', 'reasonable', 'price', 'also', 'happen', 'think', 'coat', 'shoe', 'best', 'item', 'color', 'offer', 'coat', 'really', 'different', 'run', 'mill', 'winter', 'peacoat', 'shoe', 'really', 'comfortable', 'durable']
['j', 'crew', 'yet', 'another', 'store', 'suburb', 'clothing', 'classy', 'style', 'love', 'j', 'crew', 'take', 'classic', 'look', 'add', 'accent', 'piece', 'make', 'look', 'oh', 'trendy', 'love', 'shopping', 'place', 'everything', 'organized', 'upbeat', 'j', 'crew', 'store', 'person', 'online', 'meet', 'need', 'day', 'work', 'weekend', 'picnic']
['much', 'love', 'clothing', 'j.crew', 'customer', 'service', 'helpful', 'time', 'shopped', "n't", 'get', 'help', 'employee', 'asked', 'pair', 'shoe', 'never', 'showed', 'shoe', 'item', 'hold', 'cash', 'register', 'returned', 'rack', 'really', 'disappointed']
['great', 'sale', 'standard', 'piece', 'picked', 'cheap', 'go', 'sale', 'everyth

['quick', 'turn-around', 'great', 'sale', 'great', 'product', '’', 'shopping', 'j.', 'crew', 'year', '’', 'planning', 'stopping', 'anytime', 'soon', 'good', 'quality', 'clothing', 'shoe', 'last', 'always', 'recommend', 'j.', 'crew', 'friend', 'looking', 'something', 'little', 'nicer', 'work', 'fun']
['quality', 'clothing', 'amazing', 'think', 'item', 'tad', 'priced', 'sale', 'section', 'great', 'well', 'j.', 'crew', 'factory', 'would', 'rather', 'pay', 'something', 'going', 'last', 'like', 'j.crew', 'everything', 'professional', 'classy', 'yet', 'cute']
['jcrew', 'great', 'place', 'buy', 'basic', 'piece', 'easy', 'return', 'store', 'mall', 'pretty', 'much', 'brand', 'really', 'got', 'facelift', '10', 'year', 'ago', 'started', 'better', 'better', 'almost', 'going', 'name', 'brand', 'luxury', 'quality', 'item', 'awesome', 'job']
['j.crew', "'s", 'product', 'tailored', 'towards', 'preppy', 'older', 'woman', 'man', 'often', 'time', 'find', 'many', 'product', 'like', '24', 'year', 'old', 'e

['giving', 'j.', 'crew', '4', 'star', 'rating', 'strictly', 'price', 'bargain', 'hunter', 'load', 'outlet', 'store', 'much', 'regular', 'department', 'store', 'clothes', 'business', 'casual', 'perfect', 'work', 'also', 'offer', 'laid', 'back', 'clothes', 'short', 'tshirts', 'weekend']
['quality', 'best', 'clothes', 'gorgeous', 'stop', 'going', 'store', 'downfall', 'find', 'repetitive', 'amazing', 'jewelry', 'one', 'strongest', 'factor']
['love', 'love', 'love', 'j.crew', 'stock', 'clothing', 'made', 'wonderful', 'addition', 'professional', 'wardrobe', "n't", 'shop', 'often', 'piece', 'staple', 'lasted', 'year', 'workplace', 'wardrobe', 'highly', 'recommended']
['easy', 'navigate', 'website', 'quality', 'clothing', 'clothing', 'expensive', 'end', 'usually', 'ship', 'seems', 'made', 'well', 'last', "n't", 'shop', 'often', 'shopping', 'good', 'sale']
['love', 'sale', 'clothes', 'return', 'policy', 'great', 'received', 'clothes', 'damaged', 'immediately', 'refunded', 'money', 'even', 'retu

...

['going', 'store', 'staff', 'always', 'pretty', 'snotty', 'store', 'always', 'mess', 'prefer', 'shop', 'online', 'one', 'always', 'find', 'need', 'pretty', 'quickly']
['pretty', 'expensive', 'clothing', 'worth', 'pretty', 'penny', 'going', 'store', 'worth', 'sometimes', 'item', 'cheaper', 'factory', 'store', 'ok', 'time']
['price', 'great', 'classic', 'piece', 'get', 'lot', 'corporate', 'work', 'clothes', 'becuase', 'long', 'lasting', 'basic', 'piece', 'mix', 'match']
['love', 'store', 'fun', 'quirky', 'work', 'piece', 'staff', 'well', 'trained', 'generally', 'helpful', 'come', 'suggestion', 'helping', 'customer', 'order', 'thing', 'store', 'shipped', 'home', 'offering', 'alternative']
['jcrew', 'offer', 'high', 'quality', 'product', 'reasonable', 'price', 'expensive', 'side', 'get', "'re", 'paying', 'clothing', 'last', 'good', 'long', 'time']
['buy', 'jean', 'jcrew', 'best', 'fit', 'love', 'wash', 'come', 'wish', 'came', 'little', 'longer', 'always', 'hit', 'ankle', 'still', 'love']
[

['prefer', 'shop', 'website', 'receive', 'clothes', 'tried', 'store', 'love', 'selection', 'shop', 'size']
['love', 'style', 'selection', 'price', 'sometimes', 'bit', 'high', 'quality', 'comparison', 'retailer', 'price', 'point']
['time', 'i️', 'slowly', 'started', 'love', 'j.', 'crew', 'much', 'i️', 'feel', 'quality', 'dwindled', 'bit', 'really', 'gotten', 'better']
['think', 'quality', 'possible', 'would', 'def', 'go', 'store', 'factory', 'store', 'bc', "'ll", 'usually', 'honor', 'student', 'discount']
['love', 'j', 'crew', 'clothes', 'cute', 'fashionable', 'wish', 'price', 'slightly', 'cheaper', 'love', 'sale', 'section']
['typically', 'great', 'customer', 'service', 'willing', 'help', 'love', 'jcrew', 'clothing', 'course', 'customer', 'service', 'make', 'difference']
['j', 'crew', 'really', 'nice', 'casual', 'wear.there', 'woman', 'coat', 'really', 'nice', "'s", 'pricey', 'side', 'worth', 'money', 'material', 'great']
['great', 'easy', 'maneuver', 'website', "'ve", 'never', 'proble

['love', 'kid', 'line', 'husband', 'love', 'slack', 'work', 'quality', 'great', 'clothes', 'never', 'go', 'style']
['love', 'j.crew', 'ipad', 'case', 'makeup', 'bag', 'style', 'modern', 'feel', 'eye', 'catching']
['best', 'coat', 'wait', 'sale', 'great', 'coat', 'best', 'quality', '’', 'love', 'forever', 'worth', 'splurge']
['quality', 'comoanies', 'material', 'best', 'pair', 'shirt', 'love', 'much', "'s", 'ridiculous']
['love', 'shopping', 'jcrew', 'online', 'site', 'easy', 'navigate', 'feel', 'like', 'always', 'deal', 'online']
['personally', 'work', 'j.', 'crew', 'nothing', 'great', 'thing', 'say', 'company', 'always', 'good', 'sale']
['great', 'product', 'bit', 'pricey', 'definitely', 'made', 'well', 'last', 'long', 'definitely', 'recommend', 'others', 'shop']
['love', 'clothes', 'made', 'last', 'worn', 'professionally', 'going', 'good', 'time']
['love', 'jcrew', 'favorite', 'chino', 'short', 'work', 'wear', 'good', 'quality', 'wish', 'affordable', 'though']
['love', 'jcrew', 'thin

['great', 'clothing', 'offer', 'discount', 'student', 'j', 'crew', 'best', 'fall', 'also']
['another', 'go', 'favorite', 'place', 'shop', 'reasonably', 'priced', 'certain', 'thing', 'well']
['favorite', 'site', 'online', 'browsing', 'item', 'always', 'high', 'quality', 'stylish']
['love', 'great', 'place', 'find', 'quality', 'clothes', 'occasion', 'one', 'family']
['great', 'quality', 'clothing', 'shop', 'right', 'fairly', 'inexpensive']
['love', 'quality', 'j', 'crew', 'everything', 'timeless', 'shipping', 'prompt', 'fast']
['great', 'sale', 'clearance', 'new', 'balance', 'j.', 'crew', 'line', 'absolute', 'favorite']
['great', 'company', 'conscious', 'sustainable', 'business', 'model', 'find', 'grear', 'sale', 'service', 'good']
['love', 'jcrew', 'since', 'remember', 'quality', 'used', 'customer', 'service', 'excellent', 'credit', 'card', 'otherwise', 'depends', 'manager', 'store', 'overall', 'still', 'good', 'company']
['jcrew', 'one', 'favourite', 'place', 'browse', 'online', 'indul

['amo', 'esa', 'página', 'e', '100', 'recomendada', 'excelente', 'de', 'verdad', 'si', 'pueden', 'probarla', 'háganlo']
5
[]


In [106]:
print((neg_reviews))

[['received', 'item', 'gift', 'tried', 'return', 'money', 'different', 'size', 'went', 'mess', 'customer', 'service', 'quality', 'clothes', 'good', 'customer', 'service'], ['shipping', 'took', 'week', 'longer', 'stated', 'checked', 'dress', 'bought', 'event', 'arrive', 'time', 'disappointed', '’', 'get', 'reply', 'email', 'probably', 'shop', 'like', 'clothing', 'good', 'sale'], ['jcrew', 'used', 'great', 'sweater', 'coat', 'find', 'lately', 'option', 'limited', 'usually', 'stuff', 'however', 'quality', 'clothing', 'justifies', 'price', 'better', 'brand', 'range'], ['j.', 'crew', 'used', 'one', 'time', 'favorite', 'brand', "'m", 'sure', 'grew', 'style', 'changing', 'last', 'year', "'m", 'finding', 'harder', 'harder', 'relate', 'style'], ['love', 'j.', 'crew', 'factory', 'store', 'bump', 'price', 'sale', 'think', 'saving', 'really', 'conned', 'cute', 'stuff', 'though', 'good', 'quality'], ['j', 'crew', 'demonstrates', 'capable', 'handle', 'business', 'appropriately', 'instead', 'think', 

In [95]:
data_words_pos = []

data_words_neg = []

for wd_list in pos_reviews:
    for word in wd_list:
        data_words_pos.append(word)
        
for wd_list in neg_reviews:
    for word in wd_list:
        data_words_neg.append(word)

In [96]:
data_words_pos = [data_words_pos]

data_words_neg = [data_words_neg]

In [97]:
# Build the bigram and trigram models
bigram_pos = gensim.models.Phrases(data_words_pos, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_pos = gensim.models.Phrases(bigram_pos[data_words_pos], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_pos = gensim.models.phrases.Phraser(bigram_pos)
trigram_mod_pos = gensim.models.phrases.Phraser(trigram_pos)

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [98]:
bigram_neg = gensim.models.Phrases(data_words_neg, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_neg = gensim.models.Phrases(bigram_neg[data_words_neg], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_neg = gensim.models.phrases.Phraser(bigram_neg)
trigram_mod_neg = gensim.models.phrases.Phraser(trigram_neg)

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [99]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams_pos(texts):
    return [bigram_mod_pos[doc] for doc in texts]

def make_trigrams_pos(texts):
    return [trigram_mod_pos[bigram_mod_pos[doc]] for doc in texts]

def make_bigrams_neg(texts):
    return [bigram_mod_neg[doc] for doc in texts]

def make_trigrams_neg(texts):
    return [trigram_mod_neg[bigram_mod_neg[doc]] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [100]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [101]:
# Remove Stop Words
data_words_pos_nostops = remove_stopwords(data_words_pos)

data_words_neg_nostops = remove_stopwords(data_words_neg)

# Form Bigrams
data_words_bigrams_pos = make_bigrams_pos(data_words_pos_nostops)

data_words_bigrams_neg = make_bigrams_neg(data_words_neg_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv

nlp.max_length = 15000000

data_lemmatized_pos = lemmatization(data_words_bigrams_pos, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_lemmatized_neg = lemmatization(data_words_bigrams_neg, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [102]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized_pos)

# Create Corpus
texts = data_lemmatized_pos

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

id2word_neg = corpora.Dictionary(data_lemmatized_neg)

# Create Corpus
texts = data_lemmatized_neg

# Term Document Frequency
corpus_neg = [id2word_neg.doc2bow(text) for text in texts]

# View
# print(corpus[:1])

In [103]:
import pprint

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model_neg = gensim.models.ldamodel.LdaModel(corpus=corpus_neg,
                                           id2word=id2word_neg,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

pprint.pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print('\n\n\n')
pprint.pprint(lda_model_neg.print_topics())
doc_lda_neg = lda_model_neg[corpus]

[(0,
  '0.002*"love" + 0.002*"great" + 0.002*"crew" + 0.002*"store" + '
  '0.001*"clothe" + 0.001*"always" + 0.001*"good" + 0.001*"quality" + '
  '0.001*"sale" + 0.001*"jcrew"'),
 (1,
  '0.032*"love" + 0.027*"quality" + 0.026*"great" + 0.023*"crew" + '
  '0.020*"store" + 0.019*"clothe" + 0.019*"good" + 0.018*"sale" + '
  '0.017*"always" + 0.016*"jcrew"'),
 (2,
  '0.004*"love" + 0.004*"quality" + 0.003*"crew" + 0.003*"good" + '
  '0.003*"store" + 0.003*"great" + 0.002*"sale" + 0.002*"always" + '
  '0.002*"jcrew" + 0.002*"clothe"'),
 (3,
  '0.003*"love" + 0.002*"great" + 0.002*"quality" + 0.002*"jcrew" + '
  '0.002*"crew" + 0.002*"store" + 0.002*"sale" + 0.002*"shop" + 0.002*"good" + '
  '0.002*"clothe"'),
 (4,
  '0.002*"love" + 0.002*"quality" + 0.002*"great" + 0.002*"clothe" + '
  '0.001*"crew" + 0.001*"store" + 0.001*"good" + 0.001*"always" + 0.001*"sale" '
  '+ 0.001*"clothing"'),
 (5,
  '0.002*"love" + 0.002*"great" + 0.002*"quality" + 0.002*"crew" + '
  '0.002*"sale" + 0.001*"good"

In [104]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x             y  topics  cluster       Freq
topic                                                    
1     -0.191158  9.523125e-09       1        1  99.998581
6      0.036405 -9.514226e-06       2        1   0.000175
0      0.031146  1.616383e-05       3        1   0.000168
4      0.030653  6.862180e-05       4        1   0.000168
5      0.029387 -2.288789e-05       5        1   0.000166
9      0.020933 -1.494986e-04       6        1   0.000157
7      0.017517  3.142914e-05       7        1   0.000153
3      0.014062 -5.307430e-06       8        1   0.000149
8      0.008007  1.716200e-04       9        1   0.000145
2      0.003048 -1.006362e-04      10        1   0.000143, topic_info=     Category        Freq       Term       Total  loglift  logprob
term                                                              
665   Default  299.000000       love  299.000000  30.0000  30.0000
904   Default  255.000000    quality  255.000000  29.0000  29.0000
505   Default  242.000000      great  242.000000  28.0000  28.0000
263   Default  219.000000       crew  219.000000  27.0000  27.0000
1092  Default  190.000000      store  190.000000  26.0000  26.0000
197   Default  180.000000     clothe  180.000000  25.0000  25.0000
494   Default  176.000000       good  176.000000  24.0000  24.0000
972   Default  167.000000       sale  167.000000  23.0000  23.0000
37    Default  159.000000     always  159.000000  22.0000  22.0000
590   Default  149.000000      jcrew  149.000000  21.0000  21.0000
1010  Default  131.000000       shop  131.000000  20.0000  20.0000
198   Default  114.000000   clothing  114.000000  19.0000  19.0000
431   Default   98.000000       find   98.000000  18.0000  18.0000
491   Default   96.000000         go   96.000000  17.0000  17.0000
1105  Default   95.000000      style   95.000000  16.0000  16.0000
871   Default   94.000000      price   94.000000  15.0000  15.0000
778   Default   85.000000     online   85.000000  14.0000  14.0000
483   Default   84.000000        get   84.000000  13.0000  13.0000
143   Default   75.000000        buy   75.000000  12.0000  12.0000
1253  Default   73.000000    website   73.000000  11.0000  11.0000
840   Default   68.000000      piece   68.000000  10.0000  10.0000
183   Default   67.000000    classic   67.000000   9.0000   9.0000
680   Default   64.000000       make   64.000000   8.0000   8.0000
587   Default   66.000000       item   66.000000   7.0000   7.0000
926   Default   64.000000     really   64.000000   6.0000   6.0000
659   Default   64.000000       look   64.000000   5.0000   5.0000
1013  Default   63.000000   shopping   63.000000   4.0000   4.0000
35    Default   63.000000       also   63.000000   3.0000   3.0000
883   Default   58.000000    product   58.000000   2.0000   2.0000
615   Default   59.000000       last   59.000000   1.0000   1.0000
...       ...         ...        ...         ...      ...      ...
494   Topic10    0.000038       good  176.669876  -1.8963  -5.8719
1092  Topic10    0.000036      store  190.474854  -2.0315  -5.9319
505   Topic10    0.000035      great  242.842819  -2.2851  -5.9426
972   Topic10    0.000032       sale  167.816605  -2.0076  -6.0346
37    Topic10    0.000032     always  159.865540  -1.9760  -6.0515
590   Topic10    0.000030      jcrew  149.934540  -1.9610  -6.1006
197   Topic10    0.000030     clothe  180.604568  -2.1544  -6.1079
1010  Topic10    0.000025       shop  131.309311  -2.0240  -6.2963
1105  Topic10    0.000023      style   95.775185  -1.7873  -6.3752
491   Topic10    0.000023         go   96.747002  -1.8021  -6.3798
840   Topic10    0.000021      piece   68.150085  -1.5457  -6.4739
143   Topic10    0.000021        buy   75.986725  -1.6442  -6.4634
431   Topic10    0.000022       find   98.693710  -1.8621  -6.4199
183   Topic10    0.000020    classic   67.168900  -1.5519  -6.4945
1253  Topic10    0.000020    website   73.078743  -1.6416  -6.4999
483   Topic10    0.000021        get   84.923

In [105]:
vis_neg = pyLDAvis.gensim.prepare(lda_model_neg, corpus_neg, id2word_neg)
vis_neg

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
2     -0.086866 -0.0       1        1  99.977585
0      0.004187  0.0       2        1   0.003061
3      0.012612 -0.0       3        1   0.002518
1      0.011323  0.0       4        1   0.002451
7      0.010990  0.0       5        1   0.002444
4      0.011015  0.0       6        1   0.002441
9      0.009516 -0.0       7        1   0.002384
8      0.009606 -0.0       8        1   0.002384
6      0.009069  0.0       9        1   0.002374
5      0.008547 -0.0      10        1   0.002369, topic_info=     Category       Freq        Term      Total  loglift  logprob
term                                                             
171   Default  45.000000        love  45.000000  30.0000  30.0000
226   Default  27.000000     quality  27.000000  29.0000  29.0000
127   Default  18.000000       great  18.000000  28.0000  28.0000
68    Default  18.000000        crew  18.000000  27.0000  27.0000
277   Default  17.000000       store  17.000000  26.0000  26.0000
53    Default  15.000000    clothing  15.000000  25.0000  25.0000
125   Default  13.000000        good  13.000000  24.0000  24.0000
218   Default  12.000000       price  12.000000  23.0000  23.0000
143   Default  12.000000        item  12.000000  22.0000  22.0000
10    Default  12.000000      always  12.000000  21.0000  21.0000
311   Default   9.000000        well   9.000000  20.0000  20.0000
52    Default   9.000000      clothe   9.000000  19.0000  19.0000
124   Default   8.000000          go   8.000000  18.0000  18.0000
259   Default   8.000000        shop   8.000000  17.0000  17.0000
280   Default   8.000000       style   8.000000  16.0000  16.0000
115   Default   8.000000        find   8.000000  15.0000  15.0000
244   Default   8.000000        sale   8.000000  14.0000  14.0000
145   Default   7.000000       jcrew   7.000000  13.0000  13.0000
111   Default   7.000000    favorite   7.000000  12.0000  12.0000
232   Default   7.000000      really   7.000000  11.0000  11.0000
294   Default   6.000000        time   6.000000  10.0000  10.0000
31    Default   6.000000         buy   6.000000   9.0000   9.0000
122   Default   5.000000         get   5.000000   8.0000   8.0000
135   Default   5.000000        high   5.000000   7.0000   7.0000
169   Default   5.000000        look   5.000000   6.0000   6.0000
87    Default   5.000000       dress   5.000000   5.0000   5.0000
303   Default   5.000000         use   5.000000   4.0000   4.0000
24    Default   5.000000       brand   5.000000   3.0000   3.0000
194   Default   4.000000       often   4.000000   2.0000   2.0000
255   Default   4.000000     service   4.000000   1.0000   1.0000
...       ...        ...         ...        ...      ...      ...
277   Topic10   0.000072       store  17.422836  -1.7523  -5.6099
127   Topic10   0.000072       great  18.376629  -1.8027  -5.6070
226   Topic10   0.000071     quality  27.039978  -2.2034  -5.6214
68    Topic10   0.000069        crew  18.388384  -1.8372  -5.6409
53    Topic10   0.000069    clothing  15.493131  -1.6759  -5.6508
218   Topic10   0.000067       price  12.602684  -1.4874  -5.6689
124   Topic10   0.000066          go   8.751761  -1.1412  -5.6874
125   Topic10   0.000067        good  13.566826  -1.5638  -5.6715
52    Topic10   0.000066      clothe   9.718418  -1.2446  -5.6859
10    Topic10   0.000066      always  12.607191  -1.5061  -5.6872
115   Topic10   0.000065        find   8.759271  -1.1556  -5.7009
259   Topic10   0.000065        shop   8.753142  -1.1562  -5.7022
311   Topic10   0.000065        well   9.717174  -1.2632  -5.7047
194   Topic10   0.000064       often   4.907270  -0.6031  -5.7277
145   Topic10   0.000064       jcrew   7.793536  -1.0563  -5.7184
232   Topic10   0.000064      really   7.796777  -1.0585  -5.7202
143   Topic10   0.000065        item  12.602702  -1.5273  -5.7087
303   Topic10   0.000063         use   5.869757  -0.7856  -5.7311
122

[  Top Competitors  Websites Market Share             Versus Page
 0          Stripe      3793       72.29%       PayPal vs. Stripe
 1          Klarna       303        5.77%       PayPal vs. Klarna
 2         GiroPay        42         0.8%      PayPal vs. GiroPay
 3     PaySafeCard        34        0.65%  PayPal vs. PaySafeCard
 4         Paysafe        15        0.29%      PayPal vs. Paysafe]